In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
def luo_yhteys():
    try:
        connection = psycopg2.connect(
            host="192.168.0.15",
            database="VehicleRegisterDatabase",
            user="postgres",
            password="1337"
        )
        #print("Connection to PostgreSQL database successful")
        print(connection)
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL database:", error)

    try:
        cursor = connection.cursor()
        print("Cursor created successfully")
    except (Exception, psycopg2.Error) as error:
        print("Error while creating a cursor:", error)

    return cursor


In [16]:
cursor = luo_yhteys()
query = f'''
    SELECT kunta, ajoneuvoluokka, ensirekisterointipvm
    FROM ajoneuvorekisteroinnit
    WHERE kunta = '179' AND ajoneuvoluokka = 'M1' AND ensirekisterointipvm >= '2010-01-01' AND ensirekisterointipvm <= '2022-01-01'
    ORDER BY ensirekisterointipvm
'''
try:
    cursor.execute(query)
    # Fetch and print the results
    records = cursor.fetchall()
    df = pd.DataFrame(records, columns=["kunta", "ajoneuvoluokka", "ensirekisterointopvm"])
    print(df)

    # Commit the transaction
    #connection.commit()
except (Exception, psycopg2.Error) as error:
    print("Error while executing SQL query:", error)


<connection object at 0x000001833404D8B0; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
       kunta ajoneuvoluokka ensirekisterointopvm
0        179             M1           2010-01-02
1        179             M1           2010-01-02
2        179             M1           2010-01-02
3        179             M1           2010-01-04
4        179             M1           2010-01-04
...      ...            ...                  ...
20276    179             M1           2020-05-07
20277    179             M1           2020-08-20
20278    179             M1           2021-05-18
20279    179             M1           2021-07-23
20280    179             M1           2021-11-01

[20281 rows x 3 columns]


In [36]:
def ensirekisteroinnit_kunnittain_ajoneuvoluokassa(pvm_alku, pvm_loppu, ajoneuvoluokka, kunta):
    cursor = luo_yhteys()
    query = f'''
        SELECT ku.selite_fi, al.lyhytselite_fi, ar.ensirekisterointipvm
        FROM ajoneuvorekisteroinnit ar
        JOIN kunta ku ON ku.id = ar.kunta
        JOIN ajoneuvoluokka al ON al.id = ar.ajoneuvoluokka
        WHERE ku.selite_fi = '{kunta.title()}' AND ajoneuvoluokka = '{ajoneuvoluokka}' AND ensirekisterointipvm >= '{pvm_alku}' AND ensirekisterointipvm <= '{pvm_loppu}'
        ORDER BY ar.ensirekisterointipvm DESC
    '''
    try:
        cursor.execute(query)
        # Fetch and print the results
        records = cursor.fetchall()
        df = pd.DataFrame(records, columns=["kunta", "ajoneuvoluokka", "ensirekisterointipvm"])
        print(df)

        return df
        # Commit the transaction
        #connection.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while executing SQL query:", error)


In [37]:
ensirekisteroinnit('2010-01-01', '2022-01-01', 'M1', 'Jyväskylä')

<connection object at 0x000001833470DAD0; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
           kunta ajoneuvoluokka ensirekisterointipvm
0      Jyväskylä    Henkilöauto           2021-11-01
1      Jyväskylä    Henkilöauto           2021-07-23
2      Jyväskylä    Henkilöauto           2021-05-18
3      Jyväskylä    Henkilöauto           2020-08-20
4      Jyväskylä    Henkilöauto           2020-05-07
...          ...            ...                  ...
20276  Jyväskylä    Henkilöauto           2010-01-04
20277  Jyväskylä    Henkilöauto           2010-01-04
20278  Jyväskylä    Henkilöauto           2010-01-02
20279  Jyväskylä    Henkilöauto           2010-01-02
20280  Jyväskylä    Henkilöauto           2010-01-02

[20281 rows x 3 columns]


In [42]:
def hybridiautot_luokittain(luokka):
    cursor = luo_yhteys()
    query = f'''
        SELECT hl.lyhytselite_fi, COUNT(ar.*)
        FROM ajoneuvorekisteroinnit ar
    	LEFT JOIN hybridiluokka hl ON hl.id = ar.sahkohybridinluokka
        WHERE ar.sahkohybridi = true AND ar.ajoneuvoluokka = '{luokka}'
    	GROUP BY hl.lyhytselite_fi
    '''
    try:
        cursor.execute(query)
        # Fetch and print the results
        records = cursor.fetchall()
        df = pd.DataFrame(records, columns=["hybridityyppi", "maara"])
        df.fillna("Ei määritelty", inplace=True)
        print(df)

        return df
        # Commit the transaction
        #connection.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while executing SQL query:", error)


In [43]:
hybridiautot_luokittain('M1')

<connection object at 0x000001833470DCF0; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
                            hybridityyppi  maara
0  Pelkästään polttomoottorilla ladattava  15693
1                 Sähköverkosta ladattava   1871
2                           Ei määritelty     20


,hybridityyppi,maara
0,Pelkästään polttomoottorilla ladattava,15693
1,Sähköverkosta ladattava,1871
2,Ei määritelty,20


In [46]:
def ensirekisteroinnit_vuosittain_ominaisuuden_mukaan(ominaisuus, vuosi, luokka):
    cursor = luo_yhteys()
    query = f'''
        SELECT o.selite_fi, COUNT(ar.id) AS maara
        FROM ajoneuvorekisteroinnit ar
    	LEFT JOIN {ominaisuus} o ON o.id = ar.{ominaisuus}
        WHERE ensirekisterointipvm >= '{vuosi}-01-01' AND ensirekisterointipvm <= '{vuosi+1}-01-01' AND ar.ajoneuvoluokka = '{luokka}'
    	GROUP BY o.selite_fi
        ORDER BY maara DESC;
    '''
    try:
        cursor.execute(query)
        # Fetch and print the results
        records = cursor.fetchall()
        df = pd.DataFrame(records, columns=[ominaisuus, "maara"])
        df.fillna("Ei määritelty", inplace=True)
        print(df)

        return df
        # Commit the transaction
        #connection.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while executing SQL query:", error)

In [47]:
ensirekisteroinnit_vuosittain_ominaisuuden_mukaan('vari', 2022, 'M1')
ensirekisteroinnit_vuosittain_ominaisuuden_mukaan('kunta', 2022, 'M1')
ensirekisteroinnit_vuosittain_ominaisuuden_mukaan('ohjaamotyyppi', 2022, 'M1')
ensirekisteroinnit_vuosittain_ominaisuuden_mukaan('vaihteistotyyppi', 2022, 'M1')

<connection object at 0x000001833470F230; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
             vari  maara
0          Vihreä      8
1           Musta      8
2       Valkoinen      6
3        Punainen      6
4           Hopea      6
5  Ruskea (beige)      3
6          Harmaa      2
7         Sininen      2
8        Violetti      2
9   Ei määritelty      1
<connection object at 0x000001833470F340; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
           kunta  maara
0       Helsinki      3
1          Espoo      2
2        Kouvola      2
3         Porvoo      2
4      Seinäjoki      2
5         Kuopio      2
6      Kangasala      1
7     Kankaanpää      1
8        Kurikka      1
9          Lahti      1
10       Laitila      1
11  Lappeenranta      1
12         Lieto      1
13         Loppi      1
14       Muurame      1
15        N

,vaihteistotyyppi,maara
0,Automaattinen,22
1,Käsivalintainen,15
2,Ei määritelty,7


In [48]:
def ensirekisterointien_maarat_ajoneuvoluokittain_aikavalilla(pvm_alku, pvm_loppu):
    cursor = luo_yhteys()
    query = f'''
        SELECT al.lyhytselite_fi, COUNT(ar.id) AS maara
        FROM ajoneuvorekisteroinnit ar
    	JOIN ajoneuvoluokka al ON al.id = ar.ajoneuvoluokka
        WHERE ensirekisterointipvm >= '{pvm_alku}' AND ensirekisterointipvm <= '{pvm_loppu}'
        GROUP BY al.lyhytselite_fi
    	ORDER BY maara DESC;
    '''
    try:
        cursor.execute(query)
        # Fetch and print the results
        records = cursor.fetchall()
        df = pd.DataFrame(records, columns=["ajoneuvoluokka", "maara"])
        df.fillna("Ei määritelty", inplace=True)
        print( df)
        return df
        
        # Commit the transaction
        #connection.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while executing SQL query:", error)

In [49]:
ensirekisterointien_maarat_ajoneuvoluokittain_aikavalilla('2015-01-01','2020-01-01')
ensirekisterointien_maarat_ajoneuvoluokittain_aikavalilla('2009-01-01','20150-01-01')

<connection object at 0x000001833470F230; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
          ajoneuvoluokka   maara
0            Henkilöauto  271686
1        Kevyt perävaunu   39442
2            Pakettiauto   30290
3              Perävaunu   18302
4               Traktori   13574
5                   Mopo    9449
6          Moottoripyörä    8356
7            Kuorma-auto    7178
8        Moottorityökone    3799
9         Maastoajoneuvo    3573
10      Liikennetraktori    2179
11            Linja-auto    1208
12       Kevyt nelipyörä     755
13             Nelipyörä     160
14            Kolmipyörä      89
15  Kolmi- tai nelipyörä      10
<connection object at 0x000001833470F780; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
          ajoneuvoluokka   maara
0            Henkilöauto  947170
1        Kevyt perävaunu  180115
2         

,ajoneuvoluokka,maara
0,Henkilöauto,947170
1,Kevyt perävaunu,180115
2,Pakettiauto,102982
3,Perävaunu,70601
4,Moottoripyörä,44620
5,Traktori,43800
6,Mopo,40131
7,Kuorma-auto,26704
8,Moottorityökone,15240
9,Maastoajoneuvo,14925
